In [ ]:
using Pkg
Pkg.activate(@__DIR__)

In [ ]:
using WAV
using Images
using ImageMagick
using ProgressMeter
using VideoIO

In [ ]:
video_file = "C. Allen _ Globetrotter (Oscilloscope Music)-J2YQD8Go_Hc.mkv"

In [ ]:
video_file = "Jerobeam Fenderson - Blocks-0KDekS4YUy4.webm"

In [ ]:
run(`ffmpeg -i $video_file -vn -ab 128k -ar 44100 -y blocks.wav`)

In [ ]:
samples, fs = wavread("blocks.wav");

In [ ]:
function add_sample!(img, c1, c2)
    img[round(Int, size(img, 1) / 2 - c2 * (size(img, 1) - 2) / 2),
        round(Int, size(img, 2) / 2 + c1 * (size(img, 2) - 2) / 2)] = 1
end

In [ ]:
if !isdir("out")
    mkdir("out")
end

In [ ]:
function render(filename, samples, sample_freq, framerate=30)
    img = zeros(Gray{Float32}, 400, 400)
    encoder = prepareencoder(RGB{N0f8}.(img), framerate=framerate)
    io = open("temp.stream", "w")
    frame = 1
    interval = round(Int, sample_freq / framerate)
    @showprogress for i in (80 * 44000):(100 * 44000)
        img .*= 0.9995
        add_sample!(img, samples[i, 1], samples[i, 2])
        if i % interval == 0
            appendencode!(encoder, io, RGB{N0f8}.(img), frame)
            frame += 1
        end
    end
    finishencode!(encoder, io)
    close(io)
    mux("temp.stream", filename, framerate)
end
        

In [ ]:
render("out/blocks.mp4", samples, fs)